## Analise logs SATD

Funcionalidades: 
    
    
    Analise do gprof 
    
        Por aproximação
               Por vídeo e configuração
                -- Funções mais utilizadas (mais chamadas) (por QP)
                   - com gráfico
                -- Funções que gastam mais tempo (por QP)


    Analise do VVC log
        
        Por aproximação 
            -- bitrate
            -- PSNR
            -- tempo gasto 
    

In [48]:
from matplotlib import pyplot as plt
from IPython.display import clear_output,display, HTML
import numpy as np
import pandas as pd
import ipywidgets as widgets
import os
import glob

In [83]:
#CONSTANTS
GPROF_DIR = "../outputTests/gprof_log/"
VVCLOG_DIR =  "../outputTests/vvc_log/"

#FUNCTIONS SYSTEM
def on_button_click(b):
    #selected_values = search_aproximations(dropdown.value, "-", GPROF_DIR)

    #for value in selected_values:
    #    value = GPROF_DIR + value
    #    print(value + '\n')

    #listando todos os videos dentro das pastas das aproximacoes do valor aproximado
    values_analised = analise(dropdown.value, GPROF_DIR, "")    
#list all matrices
def list_matrices(terminator, folder):
    items = os.listdir(folder)
    names = []
    for item in items:
        if os.path.isdir(os.path.join(folder, item)):
            prefix = item.split(terminator)[0]  
            if prefix not in names:  
                names.append(prefix)
    return names
    
#list all aproximations of a matrix researched
def search_aproximations(prefix, terminator, folder):
    items = os.listdir(folder)
    names = []
    for item in items:
        if os.path.isdir(os.path.join(folder, item)):
            prefix_item = item.split(terminator)[0]
            if prefix_item == prefix: 
                names.append(item)
    return names
    
#FUNCTIONS GPROF ANALISE

#returns the most used functions list 
def m_used_functions(file_gprof, number_of_itens):
    #aqui analisar o arquivo do gprof e retornar as mais usadas
    return void; 
    
#returns the analise
def analise(matrix, folder, config):
    #find all aproximations for the matrix
    aproximations = search_aproximations(matrix, "-",folder)
    
    #find all videos for each aproximation
    for aproximation in aproximations:
        aproximation_path = folder + aproximation + "/"
        videos = os.listdir(aproximation_path);
        
        #imprime tabela dos videos
        df = pd.DataFrame(videos, columns=[aproximation])
        display(df)

    
    
    
####    
# Dropdown
options_drop = list_matrices("-", GPROF_DIR)
dropdown = widgets.Dropdown(options=options_drop)

# Botão
botao = widgets.Button(description='Imprimir')
botao.on_click(on_button_click)

In [84]:
# Exibindo os widgets
display(HTML("<h2>Analise gprof</h2>"))
display(dropdown, botao)

Dropdown(options=('16x8', '8x16', 'Precise'), value='16x8')

Button(description='Imprimir', style=ButtonStyle())

,16x8--1
0,BasketballDrive
1,BasketballPass
2,BQMall
3,Campfire
4,FoodMarket4
5,FourPeople
6,Johnny
7,RaceHorses
8,RaceHorses2
9,RitualDance


,16x8--2
0,BasketballDrive
1,BasketballPass
2,BQMall
3,Campfire
4,FoodMarket4
5,FourPeople
6,Johnny
7,RaceHorses
8,RaceHorses2
9,RitualDance


,16x8--3
0,BasketballDrive
1,BasketballPass
2,BQMall
3,Campfire
4,FoodMarket4
5,FourPeople
6,Johnny
7,RaceHorses
8,RaceHorses2
9,RitualDance
